# Oxidamp Playground

Before we can do anything else, we must compile the oxidamp crate. Note that we have set the path to `"."` and this requires that `jupyter-notebook` is started from the oxidamp directory.

In [ ]:
:dep oxidamp = { path="." }
use oxidamp::*;

Create a new audio context to configure things against:

In [ ]:
let ctx = AudioContext::new(48000);

## Setup

To execute and explore this notepad requires [evcxr_jupyter](https://github.com/google/evcxr/tree/main/evcxr_jupyter). To install this on a new machine try:

    rustup component add rust-src
    sudo apt install jupyter-notebook cmake build-essential
    cargo install evcxr_jupyter
    evcxr_jupyter --install
    jupyter-notebook
    
For more details checkout the [excxr documentation](https://github.com/google/evcxr/tree/main/evcxr_jupyter).

## Making audio playable within the jupyter notebook

This works really well to allow interactive experiments with the oxidamp DSP code. However it will also significantly bloat the git repo for every audio sample we include because the audio it generates is uncompressed and ends up burned into the notebook file.

Use with both taste and care!

In [ ]:
:dep base64
:dep hound

pub struct AudioClip {
    content: String
}

impl AudioClip {
    pub fn evcxr_display(&self) {
        println!("EVCXR_BEGIN_CONTENT text/html\n<audio controls src=\"data:audio/wav;base64,{}\"/>\nEVCXR_END_CONTENT", self.content);
    }

    fn new_spec(sample_rate: u32, channels: u16) -> (hound::WavSpec, f32) {
        (
            hound::WavSpec {
                channels,
                sample_rate,
                bits_per_sample: 16,
                sample_format: hound::SampleFormat::Int,
            },
            32767.0
        )
    }
    
    fn from_mono(buf: &[f32], sample_rate: u32) -> Self {
        let (spec, amplitude) = Self::new_spec(sample_rate, 1);
        
        let mut wav = Vec::<u8>::new();
        
        let mut writer = hound::WavWriter::new(std::io::Cursor::new(&mut wav), spec).unwrap();
        for spl in buf {
            writer.write_sample((spl * amplitude) as i16).unwrap();
        }
        writer.finalize().unwrap();
        
        Self::from_wav(&wav)
    }
    
    fn from_stereo(lbuf: &[f32], rbuf: &[f32], sample_rate: u32) -> Self {
        let (spec, amplitude) = Self::new_spec(sample_rate, 2);
        
        let mut wav = Vec::<u8>::new();
        
        let mut writer = hound::WavWriter::new(std::io::Cursor::new(&mut wav), spec).unwrap();
        for (l, r) in std::iter::zip(lbuf, rbuf) {
            writer.write_sample((*l * amplitude) as i16).unwrap();
            writer.write_sample((*r * amplitude) as i16).unwrap();
        }
        writer.finalize().unwrap();
        
        Self::from_wav(&wav)
    }
    
    fn from_wav(wav: &[u8]) -> Self {
        Self {
            content: base64::encode(wav),
        }
    }
}

In [ ]:
fn randbuf<const L: usize>() -> [f32; L] {
    let mut seed = 1;
    let mut inbuf = [0.0_f32; L];
    for i in &mut inbuf {
        *i = frand31(&mut seed);
    }
    inbuf
}
let noise = randbuf::<88200>();

AudioClip::from_mono(&noise, 44100)

# Examples

## Cabinet Simulation

Oxidamp includes a speaker cabinet simulator. Currently it is modelled (not very accurately) on the frequency reponse of a classic speaker. Implemented as a combination of biquad filters.

In [ ]:
let mut cabsim = CabinetSimulator::default();
cabsim.setup(&ctx);

We can use the filter `stimulate()` method to get the frequency response for an 400Hz sine wave:

In [ ]:
linear2db(cabsim.stimulate(&ctx, 400))

## Karplus-Strong synthesis

In [ ]:
let mut synth = KarplusStrong::default();
synth.setup(&ctx);
synth.trigger();

In [ ]:
let mut snd = [0.0_f32; 2*48000];
synth.process(&mut snd);

AudioClip::from_mono(&snd, 48000)

## Signal Generation

In [ ]:
let mut sg = SineGenerator::default();
sg.setup(&ctx, 440, 0.7);
let mut sine = [0.0_f32; 96000];
sg.process(&mut sine);

## Drum Machine

In [ ]:
let mut dm = DrumMachine::default();
dm.setup(&ctx);
dm.set_control(&Control::BeatsPerMinute(112));
dm.set_control(&Control::Pattern(3));
let mut beats = [0.0_f32; 10*48000];
dm.process(&mut beats);

AudioClip::from_mono(&beats, 48000)

Adding reverb.

In [ ]:
let mut jcrev = Reverb::default();
//jcrev.setup(&ctx);
let mut rbeats = [0.0_f32; 10*48000];
jcrev.process(&beats, &mut rbeats);

AudioClip::from_mono(&rbeats, 48000)